In [1]:
#!pip install git+git://github.com/jaredwinick/img2vec-keras.git --user

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
assert tf.config.experimental.get_memory_growth(physical_devices[0]) == True

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
# #allocate memory on demand (not use all available)
# physical_devices = tf.config.list_physical_devices('GPU') 
# try: 
#     tf.config.experimental.set_memory_growth(physical_devices[0], True) 
# except: 
#   # Invalid device or cannot modify virtual devices once initialized. 
#     pass 

# #use not all threads
# tf.config.threading.set_intra_op_parallelism_threads(5)
# tf.config.threading.set_inter_op_parallelism_threads(5)

In [ ]:
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input

from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

import matplotlib.pyplot as plt
import itertools

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import json
from IPython.display import HTML

In [ ]:
import keras
from keras.applications.vgg16 import VGG16 #224x224
from keras.applications.vgg19 import VGG19 #224x224

from keras.applications.xception import Xception # 299x299

from keras.applications.resnet import ResNet50 #224x224
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_v2 import ResNet152V2
# from keras.applications.resnext import ResNeXt50
# from keras.applications.resnext import ResNeXt101

from keras.applications.inception_v3 import InceptionV3 #299x299
from keras.applications.inception_resnet_v2 import InceptionResNetV2 #299x299

from keras.applications.mobilenet import MobileNet #224x224
from keras.applications.mobilenet_v2 import MobileNetV2 #224x224

from keras.applications.densenet import DenseNet121 #224x224
from keras.applications.densenet import DenseNet169 #224x224
from keras.applications.densenet import DenseNet201 #224x224

from keras.applications.nasnet import NASNetMobile #331x331 
from keras.applications.nasnet import NASNetLarge #224x224

In [7]:
# preprocess_input_functions = [keras.applications.vgg16.preprocess_input, 
#             keras.applications.vgg19.preprocess_input,
#             keras.applications.xception.preprocess_input, 
#             keras.applications.resnet50.preprocess_input,
#             keras.applications.resnet50.preprocess_input, 
#             keras.applications.resnet50.preprocess_input,
#             keras.applications.resnet50.preprocess_input, 
#             keras.applications.inception_v3.preprocess_input,
#             keras.applications.inception_resnet_v2.preprocess_input, 
#             keras.applications.mobilenet.preprocess_input,
#             keras.applications.mobilenet_v2.preprocess_input, 
#             keras.applications.densenet.preprocess_input,
#             keras.applications.densenet.preprocess_input, 
#             keras.applications.densenet.preprocess_input]

In [8]:
#models = [VGG16, VGG19, Xception, ResNet50, ResNet101,ResNet152, ResNet50V2,ResNet101V2,ResNet152V2,InceptionV3,InceptionResNetV2,MobileNet, MobileNetV2, DenseNet121,DenseNet169,DenseNet201,NASNetMobile,NASNetLarge]
#ResNeXt50,ResNeXt101

In [9]:
model = ResNet50(weights='imagenet')
preprocess_input_function = keras.applications.resnet50.preprocess_input
feat_extractor = Model(inputs=model.input, outputs=model.get_layer(model.layers[-2].name).output)

In [10]:
#feat_extractor = model

In [11]:
from tqdm.auto import tqdm

def get_featureas(data_path):
    
    image_names = os.listdir(data_path)
    
    imgs_model_width, imgs_model_height = feat_extractor.input_shape[1:3]
    loaded_images = []
    for img_name in tqdm(image_names):
        original = load_img(data_path+img_name, target_size=(imgs_model_width, imgs_model_height))
        numpy_image = img_to_array(original)
        image_batch = np.expand_dims(numpy_image, axis=0)
        loaded_images.append(image_batch)
    images = np.vstack(loaded_images)
    processed_imgs = preprocess_input_function(images.copy())
    imgs_features = feat_extractor.predict(processed_imgs)
    
    return imgs_features, image_names

In [12]:
data_path = "image_data/train/"
imgs_features, image_names = get_featureas(data_path)

In [18]:
labels = pd.read_csv("train.csv")
labels_dict = dict(zip(labels["name"].values, labels["category"].values))
display(labels.head(2))

# name_df = pd.DataFrame(image_names, columns=["name"])
# name_df["category"] = name_df["name"].map(labels_dict)
# name_df.head()

,name,category
0,0.jpg,47
1,1.jpg,60


In [14]:
#name_df["category"].values

In [15]:
#imgs_features, image_names = get_featureas(data_path)

In [16]:
#imgs_features.shape

### Get features for test

In [17]:
data_path = "image_data/test/"

In [18]:
imgs_features_test, image_names_test = get_featureas(data_path)

In [19]:
def check_path(path):
    os.system("if [ ! -d " + path + " ]; then mkdir -p " + path + "; fi")

In [17]:
# check_path("features/")

In [25]:
import pickle

model_name = "resnet50"

with open('features/imgs_features_test_'+model_name+'.pickle', 'wb') as f:
            pickle.dump(imgs_features_test, f)
        
with open('features/image_names_test_'+model_name+'.pickle', 'wb') as f:
            pickle.dump(image_names_test, f)
        

with open('features/imgs_features_'+model_name+'.pickle', 'wb') as f:
            pickle.dump(imgs_features, f)
        
with open('features/image_names_'+model_name+'.pickle', 'wb') as f:
            pickle.dump(image_names, f)

In [21]:
# with open('filename.pickle', 'rb') as f:
#             file_to_load = pickle.load(f)

In [22]:
# with open('features/imgs_features_test_xception.pickle', 'wb') as f:
#             pickle.dump(imgs_features_test, f)
        
# with open('features/image_names_test_xception.pickle', 'wb') as f:
#             pickle.dump(image_names_test, f)
        

# with open('features/imgs_features_xception.pickle', 'wb') as f:
#             pickle.dump(imgs_features, f)
        
# with open('features/image_names_xception.pickle', 'wb') as f:
#             pickle.dump(image_names, f)

## Try cosine simmilarity

In [23]:
K = 5

In [8]:
import pickle
import numpy as np
import pandas as pd

In [9]:
import os
os.listdir("./features")

['imgs_features_test.pickle',
 'imgs_features_resnet50.pickle',
 'imgs_features_test_inception_v3.pickle',
 'imgs_features.pickle',
 'imgs_features_test_resnet50.pickle',
 'imgs_features_inception_v3.pickle',
 'imgs_features_1.pickle',
 'imgs_features_densenet.pickle',
 'image_names_1.pickle',
 'imgs_features_test_densenet.pickle',
 'image_names_test_resnet50.pickle',
 'imgs_features_test_xception.pickle',
 'image_names_test.pickle',
 'image_names.pickle',
 'imgs_features_xception.pickle',
 'image_names_test_densenet.pickle',
 'image_names_test_1.pickle',
 'image_names_test_inception_v3.pickle',
 'imgs_features_test_1.pickle',
 'image_names_densenet.pickle',
 'image_names_resnet50.pickle',
 'image_names_test_xception.pickle',
 'image_names_xception.pickle',
 'image_names_inception_v3.pickle']

In [12]:
model_name = "densenet"

with open('features/imgs_features_'+model_name+'.pickle', 'rb') as f:
    imgs_features_train = pickle.load(f)

with open('features/image_names_'+model_name+'.pickle', 'rb') as f:
    image_names_train = pickle.load(f)
    
with open('features/imgs_features_test_'+model_name+'.pickle', 'rb') as f:
    imgs_features_test = pickle.load(f)

with open('features/image_names_test_'+model_name+'.pickle', 'rb') as f:
    image_names_test = pickle.load(f)

In [22]:
#imgs_features_train

In [25]:
#image_names_train

In [26]:
image_names_train = image_names
imgs_features_train = imgs_features

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
import pandas as pd

In [29]:
# labels = pd.read_csv("train.csv")
# labels_dict = dict(zip(labels["name"].values, labels["category"].values))
# labels.head(2)

In [30]:
# name_df = pd.DataFrame(image_names_train, columns=["name"])
# name_df["category"] = name_df["name"].map(labels_dict)
# name_df.head()

In [31]:
#imgs_features_train

In [32]:
#image_names_train 

In [13]:
K = 5

In [14]:
from tqdm.auto import tqdm

In [20]:
def get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K):
    preds = []
    cos = cosine_similarity(imgs_features_test, imgs_features_train)
    for i in tqdm(range(len(imgs_features_test))):
        nearest_indexes = cos[i].argsort()[::-1][:K]
        categ = [[labels_dict[img]]*int(cos[i][j]*100) for img,j in zip(np.array(image_names_train)[nearest_indexes], nearest_indexes)]
        categ = [item for sublist in categ for item in sublist]
        counts = np.bincount(categ)
        prediction = np.argmax(counts)
        preds.append(prediction)
    return preds

In [16]:
# def get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K):
#     preds = []
#     cos = cosine_similarity(imgs_features_test, imgs_features_train)
#     for i in tqdm(range(len(imgs_features_test))):
#         nearest_indexes = cos[i].argsort()[::-1]
#         categ = [labels_dict[img] for img in np.array(image_names_train)[nearest_indexes[:K]]]
#         counts = np.bincount(categ)
#         prediction = np.argmax(counts)
#         preds.append(prediction)
#     return preds

In [21]:
preds = get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K)

In [23]:
old_sub = pd.read_csv("sub_knn_densenet121_with_back_K5.csv")
old_sub.head(2)

,name,category
0,2078.jpg,2
1,13964.jpg,96


In [25]:
sum(old_sub.category == submission.category)

15174

In [24]:
submission = pd.DataFrame({'name':image_names_test,'category':preds})

In [26]:
submission.to_csv("sub_knn_densenet_weighted_K5.csv", index=None)

In [39]:
def send_message(message="end", show_answer=False): #get notification for Maks in telegram
    answer = os.popen('curl -s -X POST https://api.telegram.org/bot560011676:AAEH9bFv-kUsWL_z5Unt2FPu21hS9qlMdLA/sendMessage -d chat_id=182817525 -d text="'+message+'"').read()
    if show_answer:
        print(answer)

In [40]:
send_message(message="end knn")

# Try local validation

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [2]:
labels = pd.read_csv("train.csv")
labels_dict = dict(zip(labels["name"].values, labels["category"].values))
display(labels.head(2))

,name,category
0,0.jpg,47
1,1.jpg,60


In [3]:
import os
os.listdir("./features")

['imgs_features_test.pickle',
 'imgs_features_resnet50.pickle',
 'imgs_features_test_inception_v3.pickle',
 'imgs_features.pickle',
 'imgs_features_test_resnet50.pickle',
 'imgs_features_inception_v3.pickle',
 'imgs_features_1.pickle',
 'imgs_features_densenet.pickle',
 'image_names_1.pickle',
 'imgs_features_test_densenet.pickle',
 'image_names_test_resnet50.pickle',
 'imgs_features_test_xception.pickle',
 'image_names_test.pickle',
 'image_names.pickle',
 'imgs_features_xception.pickle',
 'image_names_test_densenet.pickle',
 'image_names_test_1.pickle',
 'image_names_test_inception_v3.pickle',
 'imgs_features_test_1.pickle',
 'image_names_densenet.pickle',
 'image_names_resnet50.pickle',
 'image_names_test_xception.pickle',
 'image_names_xception.pickle',
 'image_names_inception_v3.pickle']

In [7]:
def get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K):
    preds = []
    cos = cosine_similarity(imgs_features_test, imgs_features_train)
    for i in tqdm(range(len(imgs_features_test))):
        nearest_indexes = cos[i].argsort()[::-1][:K]
        categ = [[labels_dict[img]]*int(cos[i][j]*100) for img,j in zip(np.array(image_names_train)[nearest_indexes], nearest_indexes)]
        categ = [item for sublist in categ for item in sublist]
        counts = np.bincount(categ)
        prediction = np.argmax(counts)
        preds.append(prediction)
    return preds

In [8]:
model_name = "densenet"

with open('features/imgs_features_'+model_name+'.pickle', 'rb') as f:
    imgs_features_train = pickle.load(f)

with open('features/image_names_'+model_name+'.pickle', 'rb') as f:
    image_names_train = pickle.load(f)
    
with open('features/imgs_features_test_'+model_name+'.pickle', 'rb') as f:
    imgs_features_test = pickle.load(f)

with open('features/image_names_test_'+model_name+'.pickle', 'rb') as f:
    image_names_test = pickle.load(f)

In [5]:
# name_df = pd.DataFrame(image_names_train, columns=["name"])
# name_df["category"] = name_df["name"].map(labels_dict)
# name_df.head(2)

In [9]:
preds = get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K=3)

In [10]:
submission = pd.DataFrame({'name':image_names_test,'category':preds})
submission.to_csv("submissions/sub_KNN_"+model_name+"_K3.csv", index=None)

In [11]:
model_name = "inception_v3"

with open('features/imgs_features_'+model_name+'.pickle', 'rb') as f:
    imgs_features_train = pickle.load(f)

with open('features/image_names_'+model_name+'.pickle', 'rb') as f:
    image_names_train = pickle.load(f)
    
with open('features/imgs_features_test_'+model_name+'.pickle', 'rb') as f:
    imgs_features_test = pickle.load(f)

with open('features/image_names_test_'+model_name+'.pickle', 'rb') as f:
    image_names_test = pickle.load(f)

In [12]:
preds = get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K=3)

In [13]:
submission = pd.DataFrame({'name':image_names_test,'category':preds})
submission.to_csv("submissions/sub_KNN_"+model_name+"_K3.csv", index=None)

In [14]:
model_name = "resnet50"

with open('features/imgs_features_'+model_name+'.pickle', 'rb') as f:
    imgs_features_train = pickle.load(f)

with open('features/image_names_'+model_name+'.pickle', 'rb') as f:
    image_names_train = pickle.load(f)
    
with open('features/imgs_features_test_'+model_name+'.pickle', 'rb') as f:
    imgs_features_test = pickle.load(f)

with open('features/image_names_test_'+model_name+'.pickle', 'rb') as f:
    image_names_test = pickle.load(f)

In [15]:
preds = get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K=3)

In [16]:
submission = pd.DataFrame({'name':image_names_test,'category':preds})
submission.to_csv("submissions/sub_KNN_"+model_name+"_K3.csv", index=None)

In [17]:
model_name = "xception"

with open('features/imgs_features_'+model_name+'.pickle', 'rb') as f:
    imgs_features_train = pickle.load(f)

with open('features/image_names_'+model_name+'.pickle', 'rb') as f:
    image_names_train = pickle.load(f)
    
with open('features/imgs_features_test_'+model_name+'.pickle', 'rb') as f:
    imgs_features_test = pickle.load(f)

with open('features/image_names_test_'+model_name+'.pickle', 'rb') as f:
    image_names_test = pickle.load(f)

In [18]:
preds = get_predictions(imgs_features_train, imgs_features_test, image_names_train, labels_dict, K=3)

In [19]:
submission = pd.DataFrame({'name':image_names_test,'category':preds})
submission.to_csv("submissions/sub_KNN_"+model_name+"_K3.csv", index=None)

In [ ]:
X = imgs_features_train
y = name_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [53]:
preds = get_predictions(X_train, X_test, y_train["name"].values, labels_dict, K=5)

In [52]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5563463819691578

In [54]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5539739027283511

In [43]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5302491103202847

In [45]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5284697508896797

In [47]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5284697508896797

In [37]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5782918149466192

In [35]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5652431791221827

In [33]:
sum(np.array(preds) == y_test["category"].values)/len(np.array(preds))

0.5658362989323843